# Notebook to take BLASTp _C. gigas_ output to GO slim

In [1]:
pwd

'/Users/graciecrandall/Documents/GitHub/paper-pacific.oyster-larvae/notebooks'

In [2]:
wd="/Users/graciecrandall/Documents/GitHub/paper-pacific.oyster-larvae/analyses/"

In [3]:
cd $wd

/Users/graciecrandall/Documents/GitHub/paper-pacific.oyster-larvae/analyses


# Need three files

1) BLAST output format 6: https://github.com/grace-ac/paper-pacific.oyster-larvae/blob/master/analyses/0327-cgseedblast-sprot.tab
2) 